##### Maybe we'll need to collect other few weather columns like visibility (https://github.com/ekapope/WorldWeatherOnline)

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
sys.path.append('../scripts')
from feature_transform import scrap_weather
from datetime import datetime
from meteostat import Stations, Daily, Point

# Scrapping weather data for each airport within X_train time range

In [4]:
flights = pd.read_csv('../data/flights_train.csv', 
                      parse_dates = ['flight_date']).reset_index(drop=True)
url="https://raw.githubusercontent.com/ravisurdhar/flight_delays/master/airports.csv"
us_airports=pd.read_csv(url)

In [155]:
airports = np.unique((np.append(flights['from'].unique(), flights['to'].unique())))
airport_localisation = us_airports[us_airports['IATA_CODE'].isin(airports)]

In [150]:
lats = airport_localisation['LATITUDE'].tolist()
longs = airport_localisation['LONGITUDE'].tolist()
iatas = airports.tolist()
list_geo = list(zip(lats,longs))
iatas = airports.tolist()
list_data=[]

for i in range(len(list_geo)):
    point = Point(list_geo[i][0], list_geo[i][1])
    start = datetime(2011, 9, 1)
    end = datetime(2012, 11, 14)
    data = Daily(point, start, end)
    df = data.fetch()
    df = df.drop(labels=['snow', 'wdir', 'wpgt', 'pres', 'tsun'], axis=1)
    list_data.append(df.reset_index())

weather_data = pd.concat(list_data, axis=0)

In [151]:
iatas_repeated = [iata for iata in iatas for i in range(list_data[0].shape[0])]
weather_data['airport_code'] = iatas_repeated

In [147]:
weather_data.to_csv('../data/weather_data.csv')

In [171]:
weather_data

,time,tavg,tmin,tmax,prcp,wspd,airport_code
0,2011-09-01,27.8,21.7,33.9,0.0,4.0,ATL
1,2011-09-02,27.1,19.4,35.0,0.0,3.5,ATL
2,2011-09-03,27.7,20.6,35.0,0.0,5.6,ATL
3,2011-09-04,23.7,20.6,26.7,6.1,4.4,ATL
4,2011-09-05,23.7,22.2,25.6,16.0,10.8,ATL
...,...,...,...,...,...,...,...
436,2012-11-10,11.6,8.9,15.0,0.0,16.9,SFO
437,2012-11-11,10.6,6.1,15.6,0.0,3.6,SFO
438,2012-11-12,11.5,6.1,17.2,0.0,5.0,SFO
439,2012-11-13,13.5,8.3,18.3,0.0,4.3,SFO


# Merge weather data with flights data

In [179]:
weather_data = weather_data.drop(labels=['tmin','tmax'], axis=1)
weather_from = weather_data.rename(columns={'time':'flight_date', 'airport_code':'from'})
weather_from = weather_from.rename(columns={col: col+'_from' 
                        for col in df.columns if col not in ['flight_date', 'from']})

weather_to = weather_data.rename(columns={'time':'flight_date', 'airport_code':'to'})
weather_to = weather_to.rename(columns={col: col+'_to' 
                        for col in df.columns if col not in ['flight_date', 'to']})

In [180]:
flights_from = pd.merge(flights ,weather_from, on=["flight_date", "from"])

In [181]:
flights_to = pd.merge(flights_from ,weather_to, on=["flight_date", "to"])

In [182]:
flights_to

,flight_date,from,to,avg_weeks,target,std_weeks,tavg_from,prcp_from,wspd_from,tavg_to,prcp_to,wspd_to
0,2012-06-19,ORD,DFW,12.875000,12.331296,9.812647,30.0,0.0,24.8,27.5,0.0,29.2
1,2012-06-19,ORD,PHX,11.772727,10.502073,8.970490,30.0,0.0,24.8,35.4,0.0,12.2
2,2012-06-19,ORD,SFO,13.480000,12.160800,9.372477,30.0,0.0,24.8,15.7,0.0,16.9
3,2012-06-19,LAS,SFO,13.925926,10.377696,11.272587,33.8,0.0,17.6,15.7,0.0,16.9
4,2012-06-19,ORD,ATL,11.173913,11.788080,8.819669,30.0,0.0,24.8,24.6,0.0,3.6
...,...,...,...,...,...,...,...,...,...,...,...,...
8891,2012-09-16,JFK,LAS,18.973684,11.018431,11.679242,19.0,0.0,13.0,29.8,0.0,13.7
8892,2011-09-08,LAS,JFK,12.571429,10.649808,8.230155,32.0,0.0,9.4,21.8,9.7,15.5
8893,2011-09-15,BOS,PHL,7.133333,11.923892,4.688385,21.2,0.8,16.9,19.7,4.6,16.2
8894,2012-08-28,ORD,LGA,13.826087,13.316772,11.491705,23.1,0.0,8.3,26.9,1.5,19.1


# Check Missing values

In [183]:
flights_to.isnull().sum()

flight_date     0
from            0
to              0
avg_weeks       0
target          0
std_weeks       0
tavg_from      10
prcp_from       0
wspd_from       0
tavg_to        11
prcp_to         0
wspd_to         0
dtype: int64